In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import torch

In [2]:
use_cuda = torch.cuda.is_available()

In [3]:
df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [4]:
df.head()

,ID,appearedTimeOfDay,appearedHour,appearedMinute,terrainType,closeToWater,city,continent,weather,temperature,...,cooc_143,cooc_144,cooc_145,cooc_146,cooc_147,cooc_148,cooc_149,cooc_150,cooc_151,class
0,NzM4OTYzMzQxNTMyNzc1MDQ0NQ==,evening,19,10,13,False,Bangkok,Asia,Clear,27.8,...,False,False,False,False,False,False,False,False,False,2
1,NDEwMjE0MzYwNzUyMjE2OTI0NQ==,night,5,19,13,True,New_York,America,Clear,26.1,...,False,False,False,False,False,False,False,False,False,5
2,OTQ3NzU5MTA4MzgzODg2NzYxMw==,evening,19,46,0,True,New_York,America,Clear,24.7,...,False,False,False,False,False,False,False,False,False,3
3,MTU2OTA5MTcyMDMzOTc0NzczODk=,morning,11,10,0,True,Hobart,Australia,Clear,12.7,...,False,False,False,False,False,False,False,False,False,4
4,NTE2OTkxNjE3OTUzODY0MTE4MQ==,evening,18,32,13,True,Los_Angeles,America,PartlyCloudy,19.1,...,False,False,False,False,False,False,False,False,False,3


In [5]:
df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

df = df.drop(['ID'], axis=1)
targets = df['class'].values

df = df.drop(['class'], axis=1)
df = df.applymap(lambda x: 1.0 if x == True else x)
df = df.applymap(lambda x: 0.0 if x == False else x)
df = pd.get_dummies(df)
# numeric_feats = df.dtypes[df.dtypes != "object"].index
# df = df[numeric_feats]
features = df.values
df.info()
features.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7160 entries, 0 to 7159
Columns: 294 entries, appearedHour to weatherIcon_wind
dtypes: float64(177), uint8(117)
memory usage: 10.5 MB


(7160, 294)

In [6]:
df.head()

,appearedHour,appearedMinute,terrainType,closeToWater,temperature,windSpeed,pressure,population_density,urban,suburban,...,weather_WindyandFoggy,weather_WindyandPartlyCloudy,weatherIcon_clear-day,weatherIcon_clear-night,weatherIcon_cloudy,weatherIcon_fog,weatherIcon_partly-cloudy-day,weatherIcon_partly-cloudy-night,weatherIcon_rain,weatherIcon_wind
0,19.0,10.0,13.0,0.0,27.8,9.00,1008.96,6019.04440,1.0,1.0,...,0,0,1,0,0,0,0,0,0,0
1,5.0,19.0,13.0,1.0,26.1,8.70,1018.96,0.00000,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
2,19.0,46.0,0.0,1.0,24.7,16.82,1023.22,0.00000,0.0,0.0,...,0,0,1,0,0,0,0,0,0,0
3,11.0,10.0,0.0,1.0,12.7,13.25,1014.19,128.89505,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
4,18.0,32.0,13.0,1.0,19.1,5.78,1011.36,4188.39100,1.0,1.0,...,0,0,0,0,0,0,1,0,0,0


In [7]:
for k in df.columns.values:
    if (df[k].max() - df[k].min()) > 0:
        df[k] = (df[k] - df[k].min())/(df[k].max() - df[k].min())
    else:
        df[k] = 0

In [8]:
df.head()

,appearedHour,appearedMinute,terrainType,closeToWater,temperature,windSpeed,pressure,population_density,urban,suburban,...,weather_WindyandFoggy,weather_WindyandPartlyCloudy,weatherIcon_clear-day,weatherIcon_clear-night,weatherIcon_cloudy,weatherIcon_fog,weatherIcon_partly-cloudy-day,weatherIcon_partly-cloudy-night,weatherIcon_rain,weatherIcon_wind
0,0.826087,0.169492,0.8125,0.0,0.631868,0.160342,0.333774,0.601904,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.217391,0.322034,0.8125,1.0,0.585165,0.154997,0.598044,0.000000,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.826087,0.779661,0.0000,1.0,0.546703,0.299662,0.710624,0.000000,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.478261,0.169492,0.0000,1.0,0.217033,0.236059,0.471987,0.012890,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.782609,0.542373,0.8125,1.0,0.392857,0.102975,0.397199,0.418839,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [9]:
from sklearn.model_selection import train_test_split

train_ratio = 0.7
test_ratio = 0.2
# split the data into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(features, targets, test_size = 1 - train_ratio, random_state=0)
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size = test_ratio, random_state=0)
X_train.shape,X_valid.shape,X_test.shape

((5011, 294), (1719, 294), (430, 294))

In [10]:
import torch.utils.data as data_utils

data_train = data_utils.TensorDataset(torch.from_numpy(X_train).type((torch.FloatTensor)), torch.from_numpy(y_train).type((torch.LongTensor)))
data_valid = data_utils.TensorDataset(torch.from_numpy(X_valid).type((torch.FloatTensor)), torch.from_numpy(y_valid).type((torch.LongTensor)))
data_test = data_utils.TensorDataset(torch.from_numpy(X_test).type((torch.FloatTensor)), torch.from_numpy(y_test).type((torch.LongTensor)))

In [11]:
# how many data per batch to load
batch_size = 64

# convert data to torch.FloatTensor

loaders = {}
loaders['train'] = torch.utils.data.DataLoader(data_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1)

loaders['valid'] = torch.utils.data.DataLoader(data_valid,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1)
loaders['test'] = torch.utils.data.DataLoader(data_test,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1)

In [12]:
# for batch_idx, (data, target) in enumerate(loaders['train']):
#     print(target)

In [17]:
import torch.nn as nn
import torch.nn.functional as F

# define the CNN architecture
class Net(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net, self).__init__()
       
        self.fc1 = nn.Linear(294, 1024)
        self.fc2 = nn.Linear(1024, 6)


    def forward(self, x):

        x = F.relu( self.fc1(x))
        x = F.relu( self.fc2(x))
       
        return x

#-#-# You do NOT have to modify the code below this line. #-#-#

# instantiate the CNN
model = Net()

# move tensors to GPU if CUDA is available
if use_cuda:
    model.cuda()

In [14]:
import torch.optim as optim

### TODO: select loss function
criterion = nn.CrossEntropyLoss()

### TODO: select optimizer
# optimizer = optim.Adam(model.parameters(), lr=0.01)
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [15]:
import time

def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        start = time.time()
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):

#             data = data.type((torch.FloatTensor))

            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update accumulated training loss
            train_loss += loss.item()*data.size(0)
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            
#             data = data.type((torch.FloatTensor))
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()

            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update accumulated validation loss 
            valid_loss += loss.item()*data.size(0)
            

        train_loss = train_loss/len(loaders['train'].dataset)
        valid_loss = valid_loss/len(loaders['valid'].dataset)
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \t time: {:.1f}'.format(
            epoch, 
            train_loss,
            valid_loss,
            time.time() - start
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Validation loss decreased from {:.6f} to {:.6f}. Model was saved'.format(
                valid_loss_min,
                valid_loss
            ))
            
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
    
    # return trained model
    return model


# train the model
model = train(100, loaders, model, optimizer, 
                      criterion, use_cuda, 'model.pt')

Epoch: 1 	Training Loss: 26.321866 	Validation Loss: 1.791760 	 time: 0.3
Validation loss decreased from inf to 1.791760. Model was saved
Epoch: 2 	Training Loss: 1.791759 	Validation Loss: 1.791760 	 time: 0.3
Epoch: 3 	Training Loss: 1.791759 	Validation Loss: 1.791760 	 time: 0.3
Epoch: 4 	Training Loss: 1.791759 	Validation Loss: 1.791760 	 time: 0.3
Epoch: 5 	Training Loss: 1.791759 	Validation Loss: 1.791760 	 time: 0.3
Epoch: 6 	Training Loss: 1.791759 	Validation Loss: 1.791760 	 time: 0.3
Epoch: 7 	Training Loss: 1.791759 	Validation Loss: 1.791760 	 time: 0.3
Epoch: 8 	Training Loss: 1.791759 	Validation Loss: 1.791760 	 time: 0.3
Epoch: 9 	Training Loss: 1.791759 	Validation Loss: 1.791760 	 time: 0.3
Epoch: 10 	Training Loss: 1.791759 	Validation Loss: 1.791760 	 time: 0.4
Epoch: 11 	Training Loss: 1.791759 	Validation Loss: 1.791760 	 time: 0.3
Epoch: 12 	Training Loss: 1.791759 	Validation Loss: 1.791760 	 time: 0.3
Epoch: 13 	Training Loss: 1.791759 	Validation Loss: 1.7

In [16]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        
#         data = data.type((torch.FloatTensor))
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# load the model that got the best validation accuracy
model.load_state_dict(torch.load('model.pt'))
# call test function    
test(loaders, model, criterion, use_cuda)

Test Loss: 1.791760


Test Accuracy: 21% (94/430)
